In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'
manuscript_min_tf = 10
manuscript_max_tf = 0.9
state_path = 's3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev'

In [2]:
import os
from functools import partial
from itertools import groupby
from typing import List, Tuple, TypeVar

import joblib
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import data_science_pipeline.configure_warnings

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import to_gbq
from data_science_pipeline.utils.io import load_object_from
from data_science_pipeline.utils.misc import identity_fn
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query
)
from data_science_pipeline.peerscout.models import (
    WeightedKeywordModel
)

In [3]:
model_path = os.path.join(state_path, 'senior_editor_model.joblib')
recommendation_output_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_recommendation'
)

In [4]:
print('loading model from:', model_path)
model_dict = load_object_from(model_path)
model_dict.keys()

loading model from: s3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev/senior_editor_model.joblib


dict_keys(['editor_tf_idf_vectorizer', 'editor_tf_idf', 'editor_names', 'editor_person_ids'])

In [5]:
editor_tf_idf_vectorizer = model_dict['editor_tf_idf_vectorizer']
editor_tf_idf_vectorizer

TfidfVectorizer(lowercase=False, smooth_idf=False, token_pattern=None,
                tokenizer=<function identity_fn at 0x7fb5e7801dd0>)

In [6]:
editor_tf_idf = model_dict['editor_tf_idf']
editor_tf_idf

<64x19284 sparse matrix of type '<class 'numpy.float64'>'
	with 304879 stored elements in Compressed Sparse Row format>

In [7]:
editor_names = model_dict['editor_names']
editor_names

0           Aleksandra Walczak
1                  Andrew King
2               Anna Akhmanova
3     Barbara Shinn-Cunningham
4                Carla Rothlin
                ...           
59                 Tamar Makin
60             Timothy Behrens
61              Utpal Banerjee
62              Vivek Malhotra
63               Wendy Garrett
Name: name, Length: 64, dtype: object

In [8]:
editor_person_ids = model_dict['editor_person_ids']
editor_person_ids

0     50904
1     14601
2      8518
3     19576
4     44396
      ...  
59    18331
60     1044
61     1042
62     1133
63    28627
Name: person_id, Length: 64, dtype: object

In [9]:
editor_person_id_by_name_map = dict(zip(editor_names, editor_person_ids))
editor_person_id_by_name_map

{'Aleksandra Walczak': '50904',
 'Andrew King': '14601',
 'Anna Akhmanova': '8518',
 'Barbara Shinn-Cunningham': '19576',
 'Carla Rothlin': '44396',
 'Catherine Dulac': '1014',
 'Chris Baker': '28129',
 'Christian Büchel': '16197',
 'Christian Hardtke': '1102',
 'Christian Rutz': '15332',
 'Clifford Rosen': '48282',
 'Cynthia Wolberger': '5005',
 'David Ron': '1174',
 'Detlef Weigel': '1030',
 'Didier Stainier': '7189',
 'Diethard Tautz': '1191',
 'Dominique Soldati-Favre': '41647',
 'Eduardo Franco': '1086',
 'Edward Morrisey': '55645',
 'Floris de Lange': '28130',
 'Gary Westbrook': '1202',
 'George Perry': '42011',
 'Gisela Storz': '1188',
 'Huda Zoghbi': '1029',
 'James Manley': '1020',
 'Jessica Tyler': '1421',
 'John Huguenard': '13947',
 'John Kuriyan': '1018',
 'Jonathan Cooper': '1062',
 'Jos van der Meer': '41027',
 'Joshua Gold': '17965',
 'José Faraldo-Gómez': '13987',
 'K VijayRaghavan': '1027',
 'Karla Kirkegaard': '3645',
 'Kate Wassum': '33127',
 'Kathryn Cheah': '91149

In [10]:
weighted_keyword_valid_model = WeightedKeywordModel.from_tf_matrix(
    editor_tf_idf.todense(),
    editor_tf_idf_vectorizer.get_feature_names(),
    choices=editor_names
)
weighted_keyword_valid_model

In [11]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [12]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [13]:
manuscript_version_for_recommendation_df = read_big_query(
    get_sql('manuscript-version-initial-submissions-for-senior-editor-recommendation.sql').format(
        **default_query_props
    )
)
manuscript_version_for_recommendation_df.head()

> ```sql
> WITH t_manuscript_version_abstract_keywords AS (
>   SELECT
>     LPAD(CAST(manuscript_abstract_keywords.manuscript_id AS STRING), 5, '0') AS manuscript_id,
>     manuscript_abstract_keywords.version_id,
>     manuscript_abstract_keywords.extracted_keywords,
>     ROW_NUMBER() OVER (
>       PARTITION BY version_id
>       ORDER BY data_hub_imported_timestamp DESC
>     ) AS version_id_row_number
>   FROM `elife-data-pipeline.de_dev.manuscript_abstract_keywords` AS manuscript_abstract_keywords
>   WHERE ARRAY_LENGTH(extracted_keywords) > 0
> ),
> 
> t_last_manuscript_version_abstract_keywords AS (
>   SELECT
>     * EXCEPT(version_id_row_number)
>   FROM t_manuscript_version_abstract_keywords
>   WHERE version_id_row_number = 1
>   ORDER BY version_id
> )
> 
> SELECT version.version_id, manuscript_version_abstract_keywords.extracted_keywords
> FROM `elife-data-pipeline.de_dev.mv_manuscript_version` AS version
> JOIN t_last_manuscript_version_abstract_keywords AS manuscript_version_abstract_keywords
>   ON manuscript_version_abstract_keywords.version_id = version.version_id
> WHERE version.overall_stage = 'Initial Submission'
>   AND ARRAY_LENGTH(version.senior_editors) = 0
>   AND TIMESTAMP_DIFF(CURRENT_TIMESTAMP, version.created_timestamp, DAY) < 365
> ```

Downloading: 100%|██████████| 135/135 [00:01<00:00, 112.44rows/s]


,version_id,extracted_keywords
0,49069/2019-06-05T12:39:51Z,"[1 expression level, accurate, accurate model,..."
1,49224/2019-06-11T08:05:08Z,"[activity, association, bacteria, charge, char..."
2,49391/2019-06-17T03:54:44Z,"[4.1 protein, associate, associated protein, b..."
3,50111/2019-07-11T05:25:04Z,"[activity, axis, basal, basal expression, basa..."
4,51945/2019-09-17T10:10:25Z,"[activation, activation signaling, adhesion, a..."


In [14]:
keyword_similarity = cosine_similarity(
    editor_tf_idf_vectorizer.transform(
        manuscript_version_for_recommendation_df
        ['extracted_keywords']
    ),
    editor_tf_idf
)
print(keyword_similarity.max())
keyword_similarity

0.31383530383438885


array([[0.17501392, 0.09257564, 0.12925561, ..., 0.11115715, 0.11223832,
        0.12264317],
       [0.07085491, 0.05937612, 0.10026875, ..., 0.07263001, 0.10828115,
        0.11347466],
       [0.07943454, 0.05150589, 0.16412044, ..., 0.09477105, 0.18524423,
        0.11168855],
       ...,
       [0.0852068 , 0.0530061 , 0.10679407, ..., 0.07220095, 0.10555859,
        0.07373812],
       [0.10609921, 0.07532655, 0.18582608, ..., 0.20934091, 0.16618655,
        0.12729701],
       [0.05269103, 0.03297751, 0.03062724, ..., 0.0227443 , 0.02516188,
        0.02250357]])

In [15]:
weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords'][:1],
).proba_matrix

matrix([[2.03415721, 1.13798747, 1.61790829, 0.93414075, 1.02629937,
         1.38039171, 0.64326735, 1.11732231, 1.24974346, 0.7433711 ,
         1.35386394, 1.13163392, 1.63117895, 1.63379   , 1.62632737,
         1.31384611, 1.11748662, 1.53616274, 1.54702759, 0.96913994,
         1.32337975, 0.49430962, 1.37936011, 1.53648786, 1.38520956,
         1.40139857, 1.02814705, 1.13758587, 1.68245848, 1.23978909,
         1.09951496, 1.08477083, 1.41476106, 1.04157456, 1.01339855,
         1.44517484, 0.93222707, 1.58723781, 1.06902303, 1.50084782,
         1.41660841, 1.70021062, 1.36595453, 1.10551575, 1.33831071,
         2.35319153, 1.19205785, 0.99581755, 1.50253524, 1.65947917,
         0.82954672, 1.56536226, 1.23426478, 1.04119571, 1.61853927,
         1.27841102, 1.56092126, 1.35435063, 1.61372403, 0.65180527,
         1.15373103, 1.41817301, 1.41843435, 1.52320239]])

In [16]:
manuscript_matching_keywords_list = weighted_keyword_valid_model.predict_ranking(
    manuscript_version_for_recommendation_df['extracted_keywords']
).matching_keywords_list
pd.Series(manuscript_matching_keywords_list[:5])

0    [[(0.2599538292295393, cell), (0.2412992578533...
1    [[(0.120649628926658, mechanism), (0.103981531...
2    [[(0.2599538292295393, cell), (0.1279617276494...
3    [[(0.2599538292295393, cell), (0.1279617276494...
4    [[(0.241299257853316, model), (0.1279617276494...
dtype: object

In [17]:
T = TypeVar('T')


def get_recommended_editors_with_probability(
        proba_matrix: List[List[float]],
        manuscript_matching_keywords_list: List[List[Tuple[float, str]]],
        indices: List[T],
        threshold: float = 0.5) -> List[List[Tuple[float, T]]]:
    return [
        sorted([
            (
                p,
                key,
                sum(
                    s for s, _ in editor_matching_keywords
                ),
                editor_matching_keywords
            )
            for p, key, editor_matching_keywords in zip(
                row,
                indices,
                editors_matching_keywords
            ) if p >= threshold
        ], reverse=True)
        for row, editors_matching_keywords in zip(proba_matrix, manuscript_matching_keywords_list)
    ]


prediction_results_with_similarity = pd.Series(
    get_recommended_editors_with_probability(
        keyword_similarity,
        manuscript_matching_keywords_list,
        editor_names,
        threshold=0.001
    ),
    index=manuscript_version_for_recommendation_df.index
)
# print(prediction_results_with_similarity[0])
prediction_results_with_similarity[:5]

0    [(0.1989924120958892, Naama Barkai, 2.35319153...
1    [(0.15570505099555257, Philip Cole, 1.54946569...
2    [(0.25605465592054394, Dominique Soldati-Favre...
3    [(0.26097569460407843, Maureen Murphy, 2.61284...
4    [(0.22019632285697605, Richard Aldrich, 2.3775...
dtype: object

In [18]:
prediction_results_df = pd.concat([
    manuscript_version_for_recommendation_df['version_id'],
    prediction_results_with_similarity.to_frame('prediction'),
], axis=1)
print(len(prediction_results_df))
prediction_results_df.head()

135


,version_id,prediction
0,49069/2019-06-05T12:39:51Z,"[(0.1989924120958892, Naama Barkai, 2.35319153..."
1,49224/2019-06-11T08:05:08Z,"[(0.15570505099555257, Philip Cole, 1.54946569..."
2,49391/2019-06-17T03:54:44Z,"[(0.25605465592054394, Dominique Soldati-Favre..."
3,50111/2019-07-11T05:25:04Z,"[(0.26097569460407843, Maureen Murphy, 2.61284..."
4,51945/2019-09-17T10:10:25Z,"[(0.22019632285697605, Richard Aldrich, 2.3775..."


In [19]:
prediction_results_df['prediction'][0]

[(0.1989924120958892,
  'Naama Barkai',
  2.35319153134667,
  [(0.2990237762006299, 'cell'),
   (0.22079074074988342, 'model'),
   (0.12797203981467636, 'dynamic'),
   (0.12543794001636593, 'expression'),
   (0.11910269052058987, 'network'),
   (0.10228723582762961, 'level'),
   (0.08669466939049095, 'role'),
   (0.08482356141803431, 'response'),
   (0.08357615610306321, 'factor'),
   (0.07553707268663781, 'molecular'),
   (0.06018487020987255, 'type'),
   (0.05892956244935028, 'cancer'),
   (0.0584800115507042, 'mathematical'),
   (0.05337557793164327, 'prediction'),
   (0.04647688532404656, 'stress'),
   (0.043148730656296305, 'concentration'),
   (0.04307969657127719, 'disease'),
   (0.03950236024290299, 'heterogeneity'),
   (0.03917337477312496, 'variation'),
   (0.037768536343318906, 'measurement'),
   (0.0313780787067969, 'phenotypic'),
   (0.027659464922630366, 'variability'),
   (0.025609067957275845, 'shape'),
   (0.02293089706558648, 'temporal'),
   (0.022239824138184695, 'di

In [20]:
prediction_results_flat_df = pd.DataFrame([
    {
        'version_id': row.version_id,
        'score': predicted_editor[0],
        'name': predicted_editor[1],
        'person_id': editor_person_id_by_name_map[predicted_editor[1]],
        'matching_keyword_score': predicted_editor[2],
        'matching_keywords': [{
            'score': keyword_score,
            'keyword': keyword
        } for keyword_score, keyword in predicted_editor[3]],
    }
    for row in prediction_results_df.itertuples()
    for predicted_editor in row.prediction
])
print(len(prediction_results_flat_df))
prediction_results_flat_df.head()

8503


,version_id,score,name,person_id,matching_keyword_score,matching_keywords
0,49069/2019-06-05T12:39:51Z,0.198992,Naama Barkai,1725,2.353192,"[{'score': 0.2990237762006299, 'keyword': 'cel..."
1,49069/2019-06-05T12:39:51Z,0.175014,Aleksandra Walczak,50904,2.034157,"[{'score': 0.2599538292295393, 'keyword': 'cel..."
2,49069/2019-06-05T12:39:51Z,0.137961,Maureen Murphy,33764,1.700211,"[{'score': 0.2873822362113222, 'keyword': 'can..."
3,49069/2019-06-05T12:39:51Z,0.134653,Philip Cole,21020,1.659479,"[{'score': 0.2497638438649165, 'keyword': 'cel..."
4,49069/2019-06-05T12:39:51Z,0.133593,Detlef Weigel,1030,1.633790,"[{'score': 0.14850204283328203, 'keyword': 'ex..."


In [21]:
prediction_results_flat_df['version_id'].nunique()

134

In [22]:
prediction_results_flat_df.max()

version_id                58638/2020-05-06T09:35:13Z
score                                       0.313835
name                                   Wendy Garrett
person_id                                      97155
matching_keyword_score                       3.57016
dtype: object

In [23]:
print('writing to:', recommendation_output_table_name)
to_gbq(
    prediction_results_flat_df,
    recommendation_output_table_name,
    project_id=project_id,
    if_exists='replace'
)
print('done')

writing to: de_dev.data_science_editor_recommendation
done
